In [244]:
import gc
import os
import requests
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.auto import tqdm
from glob import glob
import matplotlib.pyplot as plt
from types import SimpleNamespace
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score
pd.options.display.max_colwidth = 1000
pd.options.display.max_rows = 500

In [276]:
BASE_DIR = "./data"
df_submission = pd.read_csv(f"{BASE_DIR}/submission.csv")
df_train = pd.read_csv(f"{BASE_DIR}/train.csv")
df_test = pd.read_csv(f"{BASE_DIR}/test.csv")
df_synset_meaning = pd.read_csv(f"{BASE_DIR}/synset_meaning.csv")
df_synset_meaning["word_len"] = df_synset_meaning.word.apply(len)
print(df_train.shape, df_test.shape, df_submission.shape, df_synset_meaning.shape,)

(8720, 3) (17028, 2) (17028, 2) (69, 4)


Fixing wrong labels:

`шоронгоос гармагц#0000005434 эрүүлr агаар өөдөөс нь угтан`

`эрүүлr => эрүүл`

In [277]:
df_train.loc[:, "text"] = df_train.loc[:, "text"].str.replace("r", "")

In [278]:
def preprocess_consecutive_item_error(df: pd.DataFrame, intersection_threshold=0.24):
    new_texts = []
    latest_similar = df.loc[0, "text"]
    potential_bad_dict = {}
    for i, row in tqdm(df.iterrows(), total=df.shape[0]):
        text = row.text
        tokens_prev = latest_similar.split()
        tokens_cur  = text.split()

        intersection = set(tokens_cur).intersection(tokens_prev)
        union = set(tokens_cur).union(tokens_prev)

        if len(intersection) / len(union) > intersection_threshold:
            keys = list(potential_bad_dict.keys())
            keys.reverse()

            new_text = text
            for key in keys:
                new_text =new_text.replace(key, " ".join(potential_bad_dict[key]))
            new_texts.append((i, row.text_id, new_text))
            
            df.loc[i, "text"] = new_text #     <------- Replacing
        else:
            potential_bad_dict = {}
            latest_similar = text

        query_word = None
        for tok in tokens_cur:
            if "#" in tok:
                query_word = tok.split("#")[0]
                continue
            if query_word:
                potential_bad_dict[query_word + tok] = [query_word, tok]
                break
    
    return df

In [279]:
df_train = preprocess_consecutive_item_error(df_train)
df_test = preprocess_consecutive_item_error(df_test)
df_train.shape, df_test.shape

((8720, 3), (17028, 2))

In [281]:
def replace_labeling_error(df, index=101, old="улаан,мөнгө", new="улаан, мөнгө"):
    df.loc[index, "text"] = df.loc[index, "text"].replace(old, new)
    
for a in [
    [df_train, 101, "улаан,мөнгө", "улаан, мөнгө"],
    [df_train, 666, "хугацааны шийдвэрлэх үе#0000000667 шатанд", "хугацааны шийдвэрлэх үе шатанд#0000000667"],
    [df_train, 2878, "үүднээссум", "үүднээс сум"],
    [df_train, 8547, "гэрийнбараа", "гэрийн бараа"],
    [df_train, 8714, "Cp", "сар"],
    [df_train, 8715, "Cp", "сар"],
]:
    replace_labeling_error(*a)
    
for a in [
    [df_test, 485, "хийх үе#0000009206 шатанд", "хийх үе шатанд#0000009206"],
    [df_test, 1764, "тавирч,нүд#0000010485", "тавирч, нүд#0000010485"],
    [df_test, 2353, "төвбайр#0000011074", "төв байр#0000011074"],
    [df_test, 2961, "улсcтөрийн#0000011682", "улс төрийн#0000011682"],
    [df_test, 13964, "гэрбараагаа#0000022685", "гэр бараагаа#0000022685"],
    [df_test, 10690, "төвтэйтөрийн#0000019411", "төвтэй төрийн#0000019411"],
    [df_test, 6635, "нүүр царайг#0000015356", "нүүр#0000015356 царайг"],
    
]:
    replace_labeling_error(*a)

In [282]:
import collections
# Dictionary prepration
dict_synset_meaning = collections.defaultdict(dict)
for row in df_synset_meaning.itertuples():
    dict_synset_meaning[row.word][row.synset_id] = row.meaning.lower()
synset_id2word = {row.synset_id:row.word for row in df_synset_meaning.itertuples()}
df_train["synset_word"] = df_train.synset_id.map(synset_id2word)
unique_synset = set(df_synset_meaning.word.unique().tolist())

In [283]:
# , ',', '.', '!', '?'

replace_by_empty = [
    "...", "..", "\",", '"', '%', '&', "'", '(', ')',
    '*', '+', '-', '/', ':', ';', '<', '>',
    '[', '\\', ']', '{', '}', '¬', '‘', '’', '№', "r"
]

replace_by_space = [
    '\n', 
]

def preprocess(text:str):
    # text = text.lower()
    for c in replace_by_empty:
        text = text.replace(c, "")
    for c in replace_by_space:
        text = text.replace(c, " ")
    return text

text = "Ой, модны аж үйлдвэрийн салбарт ойн баялгийг арчлан хамгаалах, зөв зохистой ашиглах тухай хууль тогтоомжийн заалтыг цөөнгүй зөрчиж байгаагаа төлөвлөгөө биелүүлэх явцад зайлшгүй тийм хохирол, зөрчил гарч#0000002121 байх юм шиг өрөөсгөлөөр ойлгож, тууштай тэмцэл хийхгүй байна"
preproces(text)

def preprocess_test(text:str):
    for c in ["...", "..", " ", "...\"", "!\""]:
        text = text.replace(c+"#", "#")
    
    return preprocess(text)
preproces_test('Өнөөдөр ажлаасаа авахуулчихлаа...#0000009274')

'Өнөөдөр ажлаасаа авахуулчихлаа#0000009274'

In [284]:
def calculate_similar_by_start(token1:str, token2:str) -> int:
    count = 0
    for i in range(min(len(token1), len(token2))):
        if token1[i] == token2[i]:
            count += 1
        else:
            break
    return count / min(len(token1), len(token2))

def find_closest(query_token):
    query_token = query_token.lower()
    if query_token in unique_synset:
        return query_token, 1.
    
    best = float("-inf")
    best_syn = None
    for syn in unique_synset:
        d = calculate_similar_by_start(query_token, syn)
        if d > best:
            best_syn = syn
            best = d
            
    return best_syn, best

In [285]:
def find_closest_meaning(text, best_syn):
    text = handle_special_char(text) # handling "#id"
    
    best = float("-inf")
    best_idx = None
    best_meaning = None
    for idx, meaning in dict_synset_meaning[best_syn].items():
        d = count_intersections(preprocess(text), preprocess(meaning))
        if d > best:
            best_idx = idx
            best_meaning = meaning
            best = d
    return best_idx, best_meaning

def handle_special_char(text): #handling #
    newtext = []
    for token in text.split():
        if "#" in token:
            token = token.split("#")[0]
        newtext.append(token)
    return " ".join(newtext)

def count_intersections(s1, s2):
    tokens1, tokens2 = set(s1.split()), set(s2.split())
    return len(tokens1.intersection(tokens2))

In [286]:
oofs = []
for i, row in tqdm(df_train.iterrows(), total=df_train.shape[0]):
    text = preprocess(row.text)
    tokens = text.split()
    
    # "query_token#00007" => get query_token
    query_tokens = [tok for tok in tokens if row.text_id[1:] in tok]
    assert len(query_tokens) == 1
    query_token = query_tokens[0].split("#")[0]
    if query_token.strip() == "":
        raise Exception(text, row.text, "\n", row)
    
    # find closest synset
    best_syn, query_score = find_closest(query_token)
    
    # find closest meaning
    best_idx, best_meaning = find_closest_meaning(text, best_syn)
    
    oofs.append((text, query_token, best_syn, query_score, best_idx, best_meaning))

df_oof = pd.DataFrame(oofs, columns=["text","query", "pred_synset", "pred_score", "pred_synset_id", "pred_meaning"])
print(accuracy_score(df_train.synset_id.values, df_oof.pred_synset_id.values))


0.756651376146789


In [287]:
df_oof = pd.concat([df_train, df_oof.drop("text", axis=1)], 1)
df_oof.head(1)

,text_id,text,synset_id,synset_word,query,pred_synset,pred_score,pred_synset_id,pred_meaning
0,t0000000001,оны долдугаар сарын#0000000001 13.,43,сар,сарын,сар,1.0,43,он цаг тоолол


In [288]:
df_oof.query("synset_word==pred_synset").shape[0] / df_oof.shape[0]

0.9997706422018349

In [289]:
df_oof.query("synset_word!=pred_synset")

,text_id,text,synset_id,synset_word,query,pred_synset,pred_score,pred_synset_id,pred_meaning
815,t0000000816,"Гэвч миний багын танил Гүрбазар зүгээр л шалбааг гатлах гэж байгаа юм шиг сарнагар хамраа сарталзуулан, нүдээ онийлгон хууч хөөрч, цуваа зургаан дугуйт#0000000816 хүчирхэг <<ЗИЛ>>-ийнхээ жолоог хэнэггүй мушгин Онон руу орж явчихав",53,тэрэг,дугуйт,баг,0.0,7,тодорхой ажил үүргийг хамтран биелүүлэхээр нэг зорилгоор нэгдсэн бүлэг хүн\nорон нутгийн засаг захиргааны хамгийн бага нэгж: багийн дарга
3590,t0000003591,"Гүнтэв гуай нэг юм гагнан, төмрийн ис, тортгонд дарагдан, хар дээрээ улам хар болчихсон юм шиг, гэвч хийн гагнуурын үе үе сацрах гэрэлд нүүр#0000003591 нь гялалзан, хөлс нь арчих завгүйг мэдсэн юм шиг хоёр хацры нь даган урсах ажээ.",56,хий,нүүр,нүүр,1.0,39,нүүр; царай


In [265]:
predictions = []
for index, row in tqdm(df_test.iterrows(), total=df_test.shape[0]):
    text = preprocess_test(row.text)
    tokens = text.split()
    
    # "query_token#00007" => get query_token
    query_tokens = [tok for tok in tokens if row.text_id[1:] in tok]
    assert len(query_tokens) == 1
    query_token = query_tokens[0].split("#")[0]
    if query_token.strip() == "":
        raise Exception(text, row.text, "\n", row)
    
    # find closest synset
    best_syn, query_score = find_closest(query_token)
    
    # find closest meaning
    best_idx, best_meaning = find_closest_meaning(text, best_syn)
    predictions.append((text, query_token, best_syn, query_score, best_idx, best_meaning))

In [266]:
df_predictions = pd.DataFrame(predictions, columns=["text","query_word", "synset_word", "score", "synset_id", "meaning"])
df_predictions = pd.concat([df_test[["text_id"]], df_predictions], axis=1)
df_test = pd.concat([df_test, df_predictions[["synset_word"]].rename({"synset_word":"pred_synset_word"})], axis=1)
print(df_predictions.shape)
df_predictions.head(2)

(17028, 7)


,text_id,text,query_word,synset_word,score,synset_id,meaning
0,t0000008721,Энэ хэсэгт оны дүгээр сарын#0000008721 2ны өдрийн хуулиар өөрчлөлт орсон,сарын,сар,1.0,43,он цаг тоолол
1,t0000008722,"Хадлангийн үеэр зарлах уралдааны болзлын өвс бэлтгэх ажлын зохистой хугацаа, өвсний чанартай шууд холбож, ажиллагчдын хөдөлмөрийн идэвхийг өрнүүлэхүйцээр хийж#0000008722 түүнийгээ хадланч бүрт одооноос сайтар ойлгуулах нь зүйтэй",хийж,хий,1.0,54,"ажил хийх, үйлдэх, хөдөлмөрлөх, ажиллах"


In [290]:
df_predictions.query("score < 1")

,text_id,text,query_word,synset_word,score,synset_id,meaning
1499,t0000010220,"Машин залуураа эргүүлэхэд жаран хөлт хорхой шиг хойд урд, дунд бүх дугуй#0000010220 нь мурийж хонгилоор могой аятай мурилзан явдаг",дугуй,баг,0.0,7,тодорхой ажил үүргийг хамтран биелүүлэхээр нэг зорилгоор нэгдсэн бүлэг хүн\nорон нутгийн засаг захиргааны хамгийн бага нэгж: багийн дарга
4083,t0000012804,"Үнэндээ уул уурхайгаас орж ирсэн мөнгийг#0000012804 хүндээ хөрөнгө оруулалт хийвэл арван жилийн дараа өнөөдөр зарсан мөнгөнөөс арав дахин их мөнгийг Монголд босгож, гаргаж ирэх чадалтай шүү дээ",мөнгийг,мөнгө,0.8,33,мөнгөний тоо; хэмжээ; мөнгө
4086,t0000012807,"Үнэндээ уул уурхайгаас орж ирсэн мөнгийг хүндээ хөрөнгө оруулалт хийвэл арван жилийн дараа өнөөдөр зарсан мөнгөнөөс арав дахин их мөнгийг#0000012807 Монголд босгож, гаргаж ирэх чадалтай шүү дээ",мөнгийг,мөнгө,0.8,33,мөнгөний тоо; хэмжээ; мөнгө


In [293]:
df_predictions[["text_id","text","query_word","synset_word","score"]]

,text_id,text,query_word,synset_word,score,synset_id,meaning
0,t0000008721,Энэ хэсэгт оны дүгээр сарын#0000008721 2ны өдрийн хуулиар өөрчлөлт орсон,сарын,сар,1.0,43,он цаг тоолол


In [295]:
df_train.head(1)

,text_id,text,synset_id,synset_word
0,t0000000001,оны долдугаар сарын#0000000001 13.,43,сар


In [294]:
df_train.to_csv("data/preprocessed/train.csv", index=False)
df_predictions[
    ["text_id","text","query_word","synset_word","score"]
].to_csv("data/preprocessed/test.csv", index=False)
df_synset_meaning.to_csv("data/preprocessed/synset_meaning.csv", index=False)
df_submission.to_csv("data/preprocessed/submission.csv", index=False)

In [296]:
import subprocess

In [300]:
process = subprocess.run([
    "kaggle",
    "competitions",
    "submit",
    "-c",
    "muis-challenge",
    "-f",
    "submission.csv",
    "-m",
    "Message"
], capture_output=True)

kaggle competitions submit -c muis-challenge -f submission.csv -m "Message"

In [304]:
print(process.stdout)

b'ref                                                        title                                          size  lastUpdated          downloadCount  voteCount  usabilityRating  \n---------------------------------------------------------  --------------------------------------------  -----  -------------------  -------------  ---------  ---------------  \nbayartsogtya/secret-kaggle-credential                      [SECRET] Kaggle Credential                     229B  2021-08-11 18:17:53              0          0  0.25             \nbayartsogtya/bayartsogt-rob-l-p-cl-ah-orig-s1000           bayartsogt-rob-l-p-cl-ah-orig-s1000             6GB  2021-07-31 21:59:41              0          0  0.3529412        \nbayartsogtya/ub-weather-2015-2020                          ulaanbaatar-weather-2015-2020                 988KB  2020-12-08 21:29:49             22          2  0.7058824        \nbayartsogtya/mlub-roberta-base-v0                          mlub-roberta-base-v0                            3